# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [216]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [217]:
sales = graphlab.SFrame('kc_house_data/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [218]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [219]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [220]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 0.646758     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 0.723130     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [221]:
# Print all coefficients
print model_all.get("coefficients").print_rows(num_rows=20, num_columns=5)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

In [222]:
# Print only with non-zero values
CoEff = model_all.get("coefficients")
coEffNonZero = CoEff[CoEff['value'] > 0]
print coEffNonZero.print_rows(num_rows=20, num_columns=5)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
+------------------+-------+---------------+
[6 rows x 3 columns]

None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [223]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [224]:
import numpy as np
l1_penalty = np.logspace(1, 7, num=13)  # [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7]
RSS = 0
DicTionary = dict()
for p in l1_penalty:
    model = graphlab.linear_regression.create(training, target='price', 
                                              features=all_features,
                                              l2_penalty=0., l1_penalty=p,
                                              verbose=False, validation_set=None)
    pOut = model.predict(validation)
    RSS = ((validation['price'] - pOut)**2).sum()
    print("l1_penalty=%e, RSS=%.2f, RSS=%e" %(p, RSS, RSS))
    DicTionary[p] = RSS

l1_penalty=1.000000e+01, RSS=625766285142459.88, RSS=6.257663e+14
l1_penalty=3.162278e+01, RSS=625766285362394.12, RSS=6.257663e+14
l1_penalty=1.000000e+02, RSS=625766286057885.00, RSS=6.257663e+14
l1_penalty=3.162278e+02, RSS=625766288257224.62, RSS=6.257663e+14
l1_penalty=1.000000e+03, RSS=625766295212186.75, RSS=6.257663e+14
l1_penalty=3.162278e+03, RSS=625766317206080.50, RSS=6.257663e+14
l1_penalty=1.000000e+04, RSS=625766386760658.12, RSS=6.257664e+14
l1_penalty=3.162278e+04, RSS=625766606749278.50, RSS=6.257666e+14
l1_penalty=1.000000e+05, RSS=625767302791634.12, RSS=6.257673e+14
l1_penalty=3.162278e+05, RSS=625769507643886.25, RSS=6.257695e+14
l1_penalty=1.000000e+06, RSS=625776517727024.00, RSS=6.257765e+14
l1_penalty=3.162278e+06, RSS=625799062845467.00, RSS=6.257991e+14
l1_penalty=1.000000e+07, RSS=625883719085425.25, RSS=6.258837e+14


In [225]:
import operator
print('#Question: What was the best value for the l1_penalty?')
print('l1_penalty=%.2f, RSS=%e' \
          % (sorted(DicTionary.items(), key=operator.itemgetter(1))[0][0], \
            sorted(DicTionary.items(), key=operator.itemgetter(1))[0][1]))

#Question: What was the best value for the l1_penalty?
l1_penalty=10.00, RSS=6.257663e+14


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [226]:
model = graphlab.linear_regression.create(training, target='price', 
                    features=all_features, l2_penalty=0., 
                    l1_penalty=sorted(DicTionary.items(), key=operator.itemgetter(1))[0][0],
                    verbose=False, validation_set=None)
RSS = 0
print('#Question: What is the RSS on TEST data of the model with the best l1_penalty?')
pOut = model.predict(testing)
RSS = ((testing['price'] - pOut)**2).sum()
l1_penalty = sorted(DicTionary.items(), key=operator.itemgetter(1))[0][0]
print("l1_penalty=%e, RSS=%.2f, RSS=%e" %(l1_penalty, RSS, RSS))

#Question: What is the RSS on TEST data of the model with the best l1_penalty?
l1_penalty=1.000000e+01, RSS=156983602381664.19, RSS=1.569836e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [227]:
# Print only with non-zero values
CoEff = model.get("coefficients")
coEffNonZero = CoEff[CoEff['value'] > 0]
print('Question: Also, using this value of L1 penalty, how many nonzero weights do you have?')
print('NonZeroCoEff=%d' %len(coEffNonZero))
print coEffNonZero.print_rows(num_rows=20, num_columns=5)

Question: Also, using this value of L1 penalty, how many nonzero weights do you have?
NonZeroCoEff=18
+------------------+-------+------------------+
|       name       | index |      value       |
+------------------+-------+------------------+
|   (intercept)    |  None |  18993.4272128   |
|     bedrooms     |  None |  7936.96767903   |
| bedrooms_square  |  None |  936.993368193   |
|    bathrooms     |  None |  25409.5889341   |
|   sqft_living    |  None |  39.1151363797   |
| sqft_living_sqrt |  None |  1124.65021281   |
|     sqft_lot     |  None | 0.00348361822299 |
|  sqft_lot_sqrt   |  None |  148.258391011   |
|      floors      |  None |   21204.335467   |
|  floors_square   |  None |  12915.5243361   |
|    waterfront    |  None |  601905.594545   |
|       view       |  None |  93312.8573119   |
|    condition     |  None |  6609.03571245   |
|      grade       |  None |  6206.93999188   |
|    sqft_above    |  None |  43.2870534193   |
|  sqft_basement   |  None |  122.

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [228]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [229]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [230]:
DictMore = dict()
DictLess = dict()
for p in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', 
                                              features=all_features,
                                              l2_penalty=0., l1_penalty=p,
                                              verbose=False, validation_set=None)
    CoEff = model.get("coefficients")
    coEffNonZero = CoEff[CoEff['value'] > 0]
    if len(coEffNonZero) > max_nonzeros:
        DictMore[p] = len(coEffNonZero)
    else:
        DictLess[p] = len(coEffNonZero)

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [231]:
l1_penalty_min = sorted(DictMore.items(), key=operator.itemgetter(1), reverse=False)[0][0]
l1_penalty_max = sorted(DictLess.items(), key=operator.itemgetter(1), reverse=True)[0][0]
l1_penalty_W_min = sorted(DictMore.items(), key=operator.itemgetter(1), reverse=False)[0][1]
l1_penalty_W_max = sorted(DictLess.items(), key=operator.itemgetter(1), reverse=True)[0][1]
print('Question: What values did you find for l1_penalty_min andl1_penalty_max?')
print('l1_penalty_min=%.f, l1_penalty_max=%.f' %(l1_penalty_min, l1_penalty_max))
print('l1_penalty_W_min=%d, l1_penalty_W_max=%d' %(l1_penalty_W_min, l1_penalty_W_max))

Question: What values did you find for l1_penalty_min andl1_penalty_max?
l1_penalty_min=2976351442, l1_penalty_max=3792690191
l1_penalty_W_min=10, l1_penalty_W_max=6


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [232]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [233]:
DictFinal = dict()
for p in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', 
                                              features=all_features,
                                              l2_penalty=0., l1_penalty=p,
                                              verbose=False, validation_set=None)
    pOut = model.predict(validation)
    RSS = ((validation['price'] - pOut)**2).sum()
    print("l1_penalty=%e, RSS=%.2f, RSS=%e" %(p, RSS, RSS))
    DictFinal[p] = RSS

l1_penalty=2.976351e+09, RSS=966925692362084.50, RSS=9.669257e+14
l1_penalty=3.019317e+09, RSS=974019450084556.12, RSS=9.740195e+14
l1_penalty=3.062282e+09, RSS=981188367942452.75, RSS=9.811884e+14
l1_penalty=3.105247e+09, RSS=989328342459474.00, RSS=9.893283e+14
l1_penalty=3.148212e+09, RSS=998783211265891.25, RSS=9.987832e+14
l1_penalty=3.191177e+09, RSS=1008477167020094.00, RSS=1.008477e+15
l1_penalty=3.234143e+09, RSS=1018298780553819.75, RSS=1.018299e+15
l1_penalty=3.277108e+09, RSS=1028247992205977.25, RSS=1.028248e+15
l1_penalty=3.320073e+09, RSS=1034616909232828.12, RSS=1.034617e+15
l1_penalty=3.363038e+09, RSS=1038554735941040.75, RSS=1.038555e+15
l1_penalty=3.406003e+09, RSS=1043237237871703.00, RSS=1.043237e+15
l1_penalty=3.448969e+09, RSS=1046937488751711.12, RSS=1.046937e+15
l1_penalty=3.491934e+09, RSS=1051147625612860.88, RSS=1.051148e+15
l1_penalty=3.534899e+09, RSS=1055992735342999.12, RSS=1.055993e+15
l1_penalty=3.577864e+09, RSS=1060799531763287.75, RSS=1.060800e+15


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [234]:
print('Question: What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzeros?')
SortedList = sorted(DictFinal.items(), key=operator.itemgetter(1))
for i in xrange(len(SortedList)):
    l1_penalty = SortedList[i][0]
    model = graphlab.linear_regression.create(training, target='price', 
                                              features=all_features,
                                              l2_penalty=0., l1_penalty=l1_penalty,
                                              verbose=False, validation_set=None)
    pOut = model.predict(validation)
    RSS = ((validation['price'] - pOut)**2).sum()
    if(RSS != SortedList[i][1]):
        print("Problem!!!")
        break
    CoEff = model.get("coefficients")
    coEffNonZero = CoEff[CoEff['value'] > 0]
    if(len(coEffNonZero) == max_nonzeros):
        print('break.. we found it..')
        break
    
print('l1_penalty=%d' % l1_penalty)
print coEffNonZero.print_rows(num_rows=20, num_columns=5)

Question: What value of l1_penalty in our narrow range has the lowest RSS on the VALIDATION set and has sparsity equal to max_nonzeros?
break.. we found it..
l1_penalty=3448968612
+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|      grade       |  None | 2899.42026975 |
|    sqft_above    |  None | 30.0115753022 |
+------------------+-------+---------------+
[7 rows x 3 columns]

None
